### Required imports

In [ ]:
from firebolt.db import connect
from firebolt.client import DEFAULT_API_URL
from datetime import datetime

### Database credentials

In [ ]:
# Only one of these two parameters should be specified
engine_url = ""
assert bool(engine_url) != bool(
    engine_name
), "Specify only one of engine_name and engine_url"

database_name = ""
username = ""
password = ""
api_endpoint = DEFAULT_API_URL

### Connecting to a database and creating cursor

In [ ]:
# create a connection based on provided credentials
connection = connect(
    engine_url=engine_url,
    engine_name=engine_name,
    database=database_name,
    username=username,
    password=password,
    api_endpoint=api_endpoint,
)

# create a cursor for connection
cursor = connection.cursor()

### Executing a query

In [ ]:
cursor.execute(
    "create fact table if not exists test_table (id int, name text, dt datetime) primary index id"
)
cursor.execute(
    "insert into test_table values (1, 'hello', '2021-01-01 01:01:01'),"
    "(2, 'world', '2022-02-02 02:02:02'),"
    "(3, '!', '2023-03-03 03:03:03')"
)

### Parameterized query

In [ ]:
cursor.execute(
    "insert into test_table values (?, ?, ?)",
    (3, "single parameter set", datetime.now()),
)
cursor.executemany(
    "insert into test_table values (?, ?, ?)",
    ((4, "multiple", datetime.now()), (5, "parameter sets", datetime.fromtimestamp(0))),
)

### Getting query description, rowcount

In [ ]:
cursor.execute("select * from test_table")
print("Description: ", cursor.description)
print("Rowcount: ", cursor.rowcount)

### Fetch query results

In [ ]:
print(cursor.fetchone())
print(cursor.fetchmany(1))
print(cursor.fetchall())

## Multi-statement queries

In [ ]:
cursor.execute(
    """
    select * from test_table where id < 4;
    select * from test_table where id > 2;
"""
)
print("First query: ", cursor.fetchall())
assert cursor.nextset()
print("Second query: ", cursor.fetchall())
assert cursor.nextset() is None

### Error handling
If one query fails during the execution, all remaining queries are canceled.
However, you still can fetch results for successful queries

In [ ]:
try:
    cursor.execute(
        """
        select * from test_table where id < 4;
        select * from test_table where wrong_field > 2;
        select * from test_table
    """
    )
except:
    pass
cursor.fetchall()

## Async interface

In [ ]:
from firebolt.async_db import connect

### Connecting to a database and creating cursor

In [ ]:
async def async_connect():
    # create a connection based on provided credentials
    connection = await connect(
        engine_url=engine_url,
        engine_name=engine_name,
        database=database_name,
        username=username,
        password=password,
        api_endpoint=api_endpoint,
    )

    # create a cursor for connection
    cursor = connection.cursor()
    return cursor

In [ ]:
async_cursor = await async_connect()

### Executing a query

In [ ]:
async def run_queries(cursor):
    await cursor.execute(
        "create fact table if not exists test_table (id int, name text, dt datetime) primary index id"
    )
    await cursor.execute(
        "insert into test_table values (1, 'hello', '2021-01-01 01:01:01'),"
        "(2, 'world', '2022-02-02 02:02:02'),"
        "(3, '!', '2023-03-03 03:03:03')"
    )
    await cursor.execute("select * from test_table")

In [ ]:
await run_queries(async_cursor)

### Getting query description, rowcount

In [ ]:
print("Description: ", async_cursor.description)
print("Rowcount: ", async_cursor.rowcount)

### Fetch query results

In [ ]:
async def print_results(cursor):
    print(await cursor.fetchone())
    print(await cursor.fetchmany(1))
    print(await cursor.fetchall())

In [ ]:
await print_results(async_cursor)

### Closing connection

In [ ]:
# manually
connection.close()

# using context manager
with connect(
    engine_url=engine_url,
    engine_name=engine_name,
    database=database_name,
    username=username,
    password=password,
    api_endpoint=api_endpoint,
) as conn:
    # create cursors, perform database queries
    pass
conn.closed